<a href="https://colab.research.google.com/github/Souleiman11/Manga_Cluster/blob/main/Copie_de_manga_scrape(worked).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
!scrapy startproject manga_scraper

New Scrapy project 'manga_scraper', using template directory '/usr/local/lib/python3.11/dist-packages/scrapy/templates/project', created in:
    /content/manga_scraper

You can start your first spider with:
    cd manga_scraper
    scrapy genspider example example.com


In [ ]:
%cd manga_scraper

/content/manga_scraper


In [ ]:
!scrapy genspider manga lekmanga.net

Created spider 'manga' using template 'basic' in module:
  manga_scraper.spiders.manga


In [ ]:
import scrapy

class MangaSpider(scrapy.Spider):
    name = "manga"
    allowed_domains = ["lekmanga.net"]
    start_urls = ["https://lekmanga.net/manga/"]

    def parse(self, response):

        # Extract manga items using correct selectors
        mangas = response.css("body > div.wrap > div > div > div.c-page-content.style-1 > div > div > div > div > div > div > div.c-page__content > div.tab-content-wrap > div > div > div:nth-child(1) > div > div:nth-child(1) > div > div.item-summary")
        for manga in mangas:
            # Extract the manga detail page URL
            relative_url = manga.css("div.post-title.font-title > h3 > a::attr(href)").get()
            if relative_url:  # Ensure URL exists
                yield response.follow(relative_url, callback=self.parse_manga_page)
        # Follow pagination link (if next page exists)
        next_page = response.css("body > div.wrap > div > div > div.c-page-content.style-1 > div > div > div > div > div > div > div.c-page__content > div.tab-content-wrap > div > nav > div > div > div > a::attr(href)").get()
        if next_page:
            yield response.follow(next_page, self.parse)


    def parse_manga_page(self, response):
        manga_name = response.css("body > div.wrap > div > div > div > div.profile-manga > div > div > div > div.post-title > h1::text").get()
        rating = response.css("body > div.wrap > div > div > div > div.profile-manga > div > div > div > div.tab-summary > div.summary_content_wrap > div > div.post-content > div.post-rating > div.post-total-rating > span::text").get()

        # Yield the extracted details
        yield {
            "manga_name": manga_name,
            "rating": rating,
            "cathegory":response.css("body > div.wrap > div > div > div > div.profile-manga > div > div > div > div.tab-summary > div.summary_content_wrap > div > div.post-content > div:nth-child(7) > div.summary-content > div > a:nth-child(1)::text").getall(),
            "stat":response.css("body > div.wrap > div > div > div > div.profile-manga > div > div > div > div.tab-summary > div.summary_content_wrap > div > div.post-status > div:nth-child(2) > div.summary-content::text").get()
        }

In [ ]:
import scrapy

class MangaSpider(scrapy.Spider):
    name = "manga"
    allowed_domains = ["lekmanga.net"]
    start_urls = ["https://lekmanga.net/manga/"]

    def parse(self, response):
        # Extract manga items using correct selectors
        mangas = response.css("div.item-summary")
        seen_mangas = set()  # Track seen manga names to avoid duplicates
        for manga in mangas:
            manga_name = manga.css("div.post-title.font-title > h3 > a::text").get()
            if manga_name not in seen_mangas:
                seen_mangas.add(manga_name)
                yield {
                    "manga_name": manga_name,
                    "rating": manga.css("div.meta-item.rating > div > span::text").get()
                }

        # Follow pagination link (if next page exists)
        next_page = response.css("body > div.wrap > div > div > div.c-page-content.style-1 > div > div > div > div > div > div > div.c-page__content > div.tab-content-wrap > div > nav > div > div > div > a::attr(href)").get()
        if next_page:
            yield response.follow(next_page, self.parse)


In [ ]:
import scrapy

class MangaSpider(scrapy.Spider):
    name = "manga"
    allowed_domains = ["lekmanga.net"]
    start_urls = ["https://lekmanga.net/manga/"]

    def parse(self, response):

        # Extract manga items using correct selectors
        mangas = response.css("body > div.wrap > div > div > div.c-page-content.style-1 > div > div > div > div > div > div > div.c-page__content > div.tab-content-wrap > div > div > div:nth-child(1) > div > div:nth-child(1) > div > div.item-summary")
        for manga in mangas:
            # Extract the manga detail page URL
            relative_url = manga.css("div.post-title.font-title > h3 > a::attr(href)").get()
            if relative_url:  # Ensure URL exists
                yield response.follow(relative_url, callback=self.parse_manga_page)
        # Follow pagination link (if next page exists)
        next_page = response.css("body > div.wrap > div > div > div.c-page-content.style-1 > div > div > div > div > div > div > div.c-page__content > div.tab-content-wrap > div > nav > div > div > div > a::attr(href)").get()
        if next_page:
            yield response.follow(next_page, self.parse)


    def parse_manga_page(self, response):
        manga_name = response.css("body > div.wrap > div > div > div > div.profile-manga > div > div > div > div.post-title > h1::text").get()
        rating = response.css("body > div.wrap > div > div > div > div.profile-manga > div > div > div > div.tab-summary > div.summary_content_wrap > div > div.post-content > div.post-rating > div.post-total-rating > span::text").get()

        # Yield the extracted details
        yield {
            "manga_name": manga_name,
            "rating": rating,
            "story":response.css("body > div.wrap > div > div > div > div.c-page-content.style-1 > div > div > div > div > div > div > div > div.description-summary > div > p::text").getall(),
            }

In [ ]:
!scrapy crawl manga -o mangas.json

2025-02-26 19:12:59 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: manga_scraper)
2025-02-26 19:12:59 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-02-26 19:12:59 [scrapy.addons] INFO: Enabled addons:
[]
2025-02-26 19:12:59 [asyncio] DEBUG: Using selector: EpollSelector
2025-02-26 19:12:59 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-02-26 19:12:59 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-02-26 19:12:59 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-02-26 19:12:59 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-